# Cassava Leaf Disease Classification — Plan

Goal: Train a robust image classifier to produce a high-accuracy submission (≥ bronze) fast, then iterate to medal.

Data available:
- train_images/ (21,367 total; 18,717 listed here implies a subset visible; train.csv maps image_id -> label [0..4])
- test_images/ (~3,000 images)
- train_tfrecords/ and test_tfrecords/ (prepared shards)
- label_num_to_disease_map.json, sample_submission.csv

Metric: Accuracy

Hardware: 1x NVIDIA A10-24Q (~24GB). Use PyTorch + timm with AMP.

Validation protocol:
- 5-fold StratifiedKFold on train.csv (deterministic seed).
- Track OOF accuracy per fold and overall.
- Save OOF logits and test logits for ensembling.

Baseline modeling (fast):
1) Model: timm tf_efficientnet_b3_ns (strong for Cassava), img size 512, pretrained=True.
2) Augmentations:
   - Light baseline: RandomResizedCrop(512), Horizontal/Vertical Flip, ColorJitter, Normalize.
   - Mixup/CutMix (alpha=0.2, p=0.5) and Label Smoothing (0.05) for stability.
3) Optimizer/Scheduler: AdamW, CosineAnnealingLR with warmup; weight_decay ~1e-4.
4) Loss: CrossEntropyLoss with label smoothing (or timm SoftTarget for mixup).
5) Training: 5 folds x 3-5 epochs for smoke baseline to verify pipeline and CV; early stop patience 1.
6) Inference: TTA (e.g., 4-8 flips/resizes) after baseline verified.

Scaling up to medal:
- Train longer (10-15 epochs) once baseline CV is solid.
- Larger/backbones for diversity:
  • tf_efficientnet_b4_ns (size 600), tf_efficientnetv2_s (size 384-448), convnext_base (size 512), seresnext50_32x4d (size 512).
- Use stronger augs: RandAugment/AutoAugment, RandomErasing, Color/Contrast jitter tuned, CutMix/Mixup balanced.
- Multiple seeds; average logits across seeds and models.
- Calibrate/ensemble via weighted average (opt weights on OOF).
- TTA at test (8-10 aug views).

Data pipeline choices:
- Prefer direct JPEG dataloader (torchvision/timm) for flexibility and on-the-fly augs. TFRecords optional; may skip.
- Ensure transforms are fit within folds; no leakage.

Efficiency:
- Use AMP, pin_memory, num_workers, persistent_workers.
- Log elapsed time per epoch/fold; save checkpoints/logits per fold.
- Start with a 10% subsample + 2 folds to sanity-check speed, then scale up.

Milestones:
M1: GPU check + EDA + fold split (OOF plan).
M2: Smoke baseline (EffNet-B3, 512, 5-fold, 3 epochs) + OOF/test logits + submission v1.
M3: Full baseline (10-12 epochs) + TTA + submission v2.
M4: Second backbone (ConvNeXt-B or EffNetV2-S) + blend + submission v3.
M5: Further diversity + seeds + optimized blend + submission v4.

Questions for Experts:
1) Backbones that consistently medal on Cassava now: still tf_efficientnet_b3/b4_ns vs effnetv2_s vs convnext_base vs seresnext50?
2) Image size sweet spot for A10-24GB vs accuracy: 512 vs 600?
3) Best augmentation recipe: RandAugment + Mixup/CutMix + LS? Recommended probabilities/strengths?
4) CV pitfalls/leakage: any particular stratification nuances for Cassava? Any grouping needed (e.g., device/source) or plain StratifiedKFold is fine?
5) TTA: how many and which transforms are most reliable here?
6) Ensembling: typical LB gain from blending 2-3 diverse models on this comp? Weighting tips?

Next steps:
- Confirm GPU works; quick EDA (class counts, image dims).
- Implement deterministic 5-fold split, dataloaders, baseline model/training loop with AMP.
- Train smoke baseline (few epochs) and generate submission v1.
- Request expert review before scaling up.

In [1]:
# Environment/GPU check + quick EDA
import sys, subprocess, os, json, random, time, glob
from pathlib import Path

def pip_install(pkg):
    try:
        __import__(pkg.split('==')[0].replace('-', '_'))
    except Exception:
        print(f"Installing {pkg}...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', pkg, '--quiet'])

# Ensure core deps
pip_install('torch')
pip_install('torchvision')
pip_install('timm')
pip_install('albumentations>=1.4.0')
pip_install('opencv-python-headless')
pip_install('pandas')
pip_install('numpy')
pip_install('Pillow')

import torch, torchvision
import timm, albumentations as A
import cv2, pandas as pd, numpy as np
from PIL import Image, ImageOps

print('GPU Available:', torch.cuda.is_available())
print('GPU Count:', torch.cuda.device_count())
if torch.cuda.is_available():
    print('GPU Name:', torch.cuda.get_device_name(0))
    props = torch.cuda.get_device_properties(0)
    print('GPU Memory (GB):', round(props.total_memory/1024**3, 1))

SEED = 42
def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed);
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed()

DATA_DIR = Path('.')
train_csv = DATA_DIR/'train.csv'
sample_csv = DATA_DIR/'sample_submission.csv'
train_dir = DATA_DIR/'train_images'
test_dir = DATA_DIR/'test_images'

df = pd.read_csv(train_csv)
print('train.csv shape:', df.shape)
print(df.head())
print('Label value_counts:')
print(df['label'].value_counts().sort_index())

train_files = sorted([p.name for p in train_dir.glob('*.jpg')])
test_files = sorted([p.name for p in test_dir.glob('*.jpg')])
print('Train images on disk:', len(train_files))
print('Test images on disk:', len(test_files))

# Check that all train image_ids exist on disk
missing = [x for x in df['image_id'].tolist() if not (train_dir/x).exists()]
print('Missing train files:', len(missing))
if len(missing) > 0:
    print('First few missing:', missing[:5])

# Sanity check sample submission columns/order
ss = pd.read_csv(sample_csv)
print('sample_submission columns:', list(ss.columns))
print('sample_submission head:')
print(ss.head())

# Probe a few image shapes and EXIF orientation handling
probe = [df['image_id'].iloc[i] for i in np.random.choice(len(df), size=min(5, len(df)), replace=False)]
for fname in probe:
    path = train_dir/fname
    try:
        with Image.open(path) as im:
            im = ImageOps.exif_transpose(im).convert('RGB')
            w, h = im.size
        arr = cv2.cvtColor(np.array(im), cv2.COLOR_RGB2BGR)
        print(f'Probe {fname}: size=({w}x{h}), dtype={arr.dtype}, shape={arr.shape}')
    except Exception as e:
        print(f'Error reading {fname}:', e)

print('ENV/EDA DONE')

Installing torch...


Installing torchvision...


Installing timm...


Installing albumentations>=1.4.0...


Installing opencv-python-headless...


Installing Pillow...


/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU Available: True
GPU Count: 1
GPU Name: NVIDIA A10-24Q
GPU Memory (GB): 23.7
train.csv shape: (18721, 2)
         image_id  label
0  1000015157.jpg      0
1  1000201771.jpg      3
2   100042118.jpg      1
3  1000723321.jpg      1
4  1000812911.jpg      3
Label value_counts:
label
0      939
1     1901
2     2091
3    11523
4     2267
Name: count, dtype: int64
Train images on disk: 18721
Test images on disk: 2676
Missing train files: 0
sample_submission columns: ['image_id', 'label']
sample_submission head:
         image_id  label
0  1234294272.jpg      4
1  1234332763.jpg      4
2  1234375577.jpg      4
3  1234555380.jpg      4
4  1234571117.jpg      4
Probe 3939876859.jpg: size=(800x600), dtype=uint8, shape=(600, 800, 3)
Probe 964482896.jpg: size=(800x600), dtype=uint8, shape=(600, 800, 3)
Probe 584853244.jpg: size=(800x600), dtype=uint8, shape=(600, 800, 3)
Probe 306133807.jpg: size=(800x600), dtype=uint8, shape=(600, 800, 3)
Probe 851527428.jpg: size=(800x600), dtype=uint8, shap

In [4]:
# Tiny overfit sanity check (no mixup/cutmix) to validate pipeline
import math
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageOps

NUM_CLASSES = 5
IMG_SIZE = 512
BATCH_SIZE = 16
NUM_WORKERS = min(8, os.cpu_count() or 2)

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

# Use robust, unambiguous resize+crop to avoid RandomResizedCrop schema issues
train_tfms = A.Compose([
    A.SmallestMaxSize(max_size=IMG_SIZE, interpolation=cv2.INTER_LINEAR),
    A.RandomCrop(height=IMG_SIZE, width=IMG_SIZE),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ColorJitter(0.2,0.2,0.2,0.1,p=0.5),
    A.RandomBrightnessContrast(0.2,0.2,p=0.5),
    A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

valid_tfms = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE, interpolation=cv2.INTER_LINEAR),
    A.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

class CassavaDS(Dataset):
    def __init__(self, df, img_dir, transforms=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = Path(img_dir)
        self.transforms = transforms
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = self.img_dir / row['image_id']
        with Image.open(path) as im:
            im = ImageOps.exif_transpose(im).convert('RGB')
            img = np.array(im)
        if self.transforms is not None:
            img = self.transforms(image=img)['image']
        # HWC->CHW
        img = torch.from_numpy(img.transpose(2,0,1)).float()
        label = int(row['label'])
        return img, torch.tensor(label, dtype=torch.long)

# Stratified tiny subset
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=SEED)
tiny_size = 200  # small to overfit quickly
df_tiny = (df.groupby('label', group_keys=False)
            .apply(lambda x: x.sample(max(1, math.ceil(tiny_size*len(x)/len(df))), random_state=SEED))
            .reset_index(drop=True))
train_idx, val_idx = next(sss.split(df_tiny['image_id'], df_tiny['label']))
df_train_tiny = df_tiny.iloc[train_idx].reset_index(drop=True)
df_valid_tiny = df_tiny.iloc[val_idx].reset_index(drop=True)
print('Tiny train/val sizes:', len(df_train_tiny), len(df_valid_tiny))

ds_tr = CassavaDS(df_train_tiny, train_dir, transforms=train_tfms)
ds_va = CassavaDS(df_valid_tiny, train_dir, transforms=valid_tfms)
dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True, persistent_workers=True)
dl_va = DataLoader(ds_va, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = timm.create_model('tf_efficientnet_b3_ns', pretrained=True, num_classes=NUM_CLASSES)
model.to(device)
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Cosine schedule with warmup (3 epochs warmup, total 8)
EPOCHS = 8
warmup_epochs = max(1, int(0.2 * EPOCHS))
total_steps = EPOCHS * max(1, len(dl_tr))
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

def cosine_warmup(step, total, warmup):
    if step < warmup:
        return step / max(1, warmup)
    progress = (step - warmup) / max(1, total - warmup)
    return 0.5 * (1 + math.cos(math.pi * progress))

global_step = 0
best_acc = 0.0
start_time = time.time()
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    t0 = time.time()
    for it, (imgs, labels) in enumerate(dl_tr):
        lr_scale = cosine_warmup(global_step, total_steps, warmup_epochs*len(dl_tr))
        for pg in optimizer.param_groups:
            pg['lr'] = 1e-3 * lr_scale
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(imgs)
            loss = criterion(logits, labels)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)
        global_step += 1
        if (it+1) % 10 == 0:
            elapsed = time.time()-t0
            print(f'Epoch {epoch+1}/{EPOCHS} | it {it+1}/{len(dl_tr)} | loss {(running_loss/total):.4f} | acc {(correct/total):.4f} | {elapsed:.1f}s', flush=True)
    train_loss = running_loss/ max(1,total)
    train_acc = correct/ max(1,total)
    # Validate
    model.eval()
    v_correct = 0
    v_total = 0
    with torch.no_grad():
        for imgs, labels in dl_va:
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            logits = model(imgs)
            preds = logits.argmax(1)
            v_correct += (preds == labels).sum().item()
            v_total += imgs.size(0)
    v_acc = v_correct / max(1, v_total)
    epoch_time = time.time()-t0
    print(f'Epoch {epoch+1} done | train_loss {train_loss:.4f} | train_acc {train_acc:.4f} | val_acc {v_acc:.4f} | epoch_time {epoch_time:.1f}s | elapsed {(time.time()-start_time)/60:.1f}m', flush=True)
    if v_acc > best_acc:
        best_acc = v_acc
        torch.save({'model': model.state_dict(), 'acc': best_acc}, 'tiny_overfit_b3_best.pth')
print('Tiny overfit best val_acc:', best_acc)
print('Sanity check complete.')

/tmp/ipykernel_60/1497617536.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(max(1, math.ceil(tiny_size*len(x)/len(df))), random_state=SEED))
/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


Tiny train/val sizes: 153 51


/tmp/ipykernel_60/1497617536.py:77: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


/tmp/ipykernel_60/1497617536.py:101: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


Epoch 1 done | train_loss 2.2376 | train_acc 0.3264 | val_acc 0.2941 | epoch_time 12.6s | elapsed 0.2m


...

Epoch 5 done | train_loss 0.6201 | train_acc 0.9514 | val_acc 0.6471 | epoch_time 2.1s | elapsed 0.4m


Epoch 6 done | train_loss 0.5628 | train_acc 0.9444 | val_acc 0.6471 | epoch_time 2.1s | elapsed 0.4m


Epoch 7 done | train_loss 0.5163 | train_acc 0.9931 | val_acc 0.6863 | epoch_time 2.1s | elapsed 0.4m


Epoch 8 done | train_loss 0.4849 | train_acc 0.9931 | val_acc 0.6863 | epoch_time 2.1s | elapsed 0.5m


Tiny overfit best val_acc: 0.7058823529411765
Sanity check complete.


In [17]:
# 1-fold sanity with timm-native transforms (Option A) + CE(LS) + EMA, explicit ImageNet mean/std
import gc, math, time, os, warnings
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
from PIL import Image, ImageOps
import numpy as np
import pandas as pd
import torch, timm
from timm.utils import ModelEmaV3

warnings.filterwarnings('ignore', category=UserWarning)
torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision('high')
except Exception:
    pass

FOLDS = 2
MAX_FOLDS = 1
EPOCHS = 3
MODEL_NAME = 'tf_efficientnet_b3_ns'
BATCH_SIZE = 24
NUM_WORKERS = min(8, os.cpu_count() or 2)
BASE_LR = 1e-4
WD = 1e-4
NUM_CLASSES = 5
IMG_SIZE = 512
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CassavaDS(Dataset):
    def __init__(self, df, img_dir, transforms=None, is_test=False):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transforms = transforms
        self.is_test = is_test
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.img_dir, row['image_id'])
        try:
            with Image.open(path) as im:
                im = ImageOps.exif_transpose(im).convert('RGB')
        except Exception:
            ridx = np.random.randint(0, len(self.df))
            rpath = os.path.join(self.img_dir, self.df.iloc[ridx]['image_id'])
            with Image.open(rpath) as im:
                im = ImageOps.exif_transpose(im).convert('RGB')
        img = im
        if self.transforms is not None:
            img = self.transforms(img)
        if self.is_test:
            return img, row['image_id']
        else:
            label = int(row['label'])
            return img, torch.tensor(label, dtype=torch.long)

def get_model_and_transforms():
    model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=NUM_CLASSES).to(device)
    cfg = timm.data.resolve_data_config({}, model=model)
    cfg['input_size'] = (3, IMG_SIZE, IMG_SIZE)
    # Critical override
    cfg['mean'] = IMAGENET_MEAN
    cfg['std'] = IMAGENET_STD
    crop_pct = float(cfg.get('crop_pct', 0.875))
    print('cfg:', {'input_size': cfg['input_size'], 'crop_pct': crop_pct, 'interpolation': cfg.get('interpolation','bicubic'), 'mean': cfg['mean'], 'std': cfg['std']})
    train_tfms = timm.data.create_transform(
        is_training=True,
        **cfg,
        auto_augment='rand-m9-mstd0.5-inc1',
        re_prob=0.25,
        re_mode='pixel'
    )
    valid_tfms = timm.data.create_transform(is_training=False, **cfg)
    return model, train_tfms, valid_tfms

def train_one_epoch(model, ema, loader, optimizer, criterion, scaler, epoch, total_steps, global_step):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    t0 = time.time()
    warmup_steps = max(1, int(0.05 * total_steps))
    for it, (imgs, labels) in enumerate(loader):
        step = global_step + it
        if step < warmup_steps:
            lr_scale = step / warmup_steps
        else:
            progress = (step - warmup_steps) / max(1, (total_steps - warmup_steps))
            lr_scale = 0.5 * (1 + math.cos(math.pi * progress))
        for pg in optimizer.param_groups:
            pg['lr'] = BASE_LR * lr_scale

        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
            logits = model(imgs)
            loss = criterion(logits, labels)
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        if ema is not None:
            ema.update(model)

        running_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += imgs.size(0)

        if (it+1) % 50 == 0:
            print(f'  it {it+1}/{len(loader)} | loss {running_loss/max(1,total):.4f} | acc {correct/max(1,total):.4f} | elapsed {time.time()-t0:.1f}s', flush=True)
    tr_loss = running_loss/max(1,total)
    tr_acc = (correct/max(1,total))
    return tr_loss, tr_acc, global_step + len(loader)

def validate(model_eval, loader):
    model_eval.eval()
    correct = 0
    total = 0
    logits_all = []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model_eval(imgs)
            logits_all.append(logits.float().cpu().numpy())
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += imgs.size(0)
    return correct/max(1,total), np.concatenate(logits_all, axis=0)

def infer_test(model_eval, loader):
    model_eval.eval()
    logits_all = []
    with torch.no_grad():
        for imgs, _ids in loader:
            imgs = imgs.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model_eval(imgs)
            logits_all.append(logits.float().cpu().numpy())
    return np.concatenate(logits_all, axis=0)

# Prepare dataframes
test_files = pd.DataFrame({'image_id': sorted([p.name for p in test_dir.glob('*.jpg')])})
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
df_shuf = df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)

oof_preds = np.zeros((len(df_shuf), NUM_CLASSES), dtype=np.float32)
test_logits_accum = np.zeros((len(test_files), NUM_CLASSES), dtype=np.float32)

fold_idx = 0
t_start_all = time.time()
for tr_idx, va_idx in skf.split(df_shuf['image_id'], df_shuf['label']):
    t_fold = time.time()
    fold_idx += 1
    print(f'===== Fold {fold_idx}/{FOLDS} | train {len(tr_idx)} | valid {len(va_idx)} =====', flush=True)
    df_tr = df_shuf.iloc[tr_idx].reset_index(drop=True)
    df_va = df_shuf.iloc[va_idx].reset_index(drop=True)

    model, train_tfms, valid_tfms = get_model_and_transforms()

    ds_tr = CassavaDS(df_tr, str(train_dir), transforms=train_tfms)
    ds_va = CassavaDS(df_va, str(train_dir), transforms=valid_tfms)
    dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_va = DataLoader(ds_va, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)

    test_ds = CassavaDS(test_files, str(test_dir), transforms=valid_tfms, is_test=True)
    test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)

    maj_class = int(df_shuf['label'].mode()[0])
    maj_acc = float((df_va['label'] == maj_class).mean())
    print(f'Majority class={maj_class} | majority baseline acc on val={maj_acc:.4f}', flush=True)

    ema = ModelEmaV3(model, decay=0.999)
    criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.05)
    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WD)
    scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())

    # Diagnostics: check val batch channel stats post-transform
    with torch.no_grad():
        imgs_chk, labels_chk = next(iter(dl_va))
        m = imgs_chk.mean(dim=[0,2,3])
        s = imgs_chk.std(dim=[0,2,3])
        print('val batch mean:', [round(float(x),4) for x in m])
        print('val batch std :', [round(float(x),4) for x in s])

    best_acc = 0.0
    best_state = None
    total_steps = EPOCHS * len(dl_tr)
    global_step = 0

    for epoch in range(1, EPOCHS+1):
        tr_loss, tr_acc, global_step = train_one_epoch(model, ema, dl_tr, optimizer, criterion, scaler, epoch, total_steps, global_step)
        eval_model = ema.module if ema is not None else model
        va_acc, _ = validate(eval_model, dl_va)
        print(f'Fold {fold_idx} | Epoch {epoch}/{EPOCHS} | tr_loss {tr_loss:.4f} tr_acc {tr_acc:.4f} | va_acc {va_acc:.4f} | elapsed_fold {(time.time()-t_fold)/60:.1f}m', flush=True)
        if va_acc > best_acc:
            best_acc = va_acc
            best_state = {k: v.cpu() for k, v in eval_model.state_dict().items()}

    eval_model = ema.module if ema is not None else model
    eval_model.eval()
    with torch.no_grad():
        for imgs, labels in dl_va:
            imgs = imgs.to(device)
            logits = eval_model(imgs)
            preds = logits.argmax(1).cpu().numpy().tolist()
            labs = labels.numpy().tolist()
            print('val preds[:16]:', preds[:16])
            print('val labs [:16]:', labs[:16])
            break

    if best_state is not None:
        eval_model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=NUM_CLASSES).to(device)
        eval_model.load_state_dict(best_state, strict=True)
    else:
        eval_model = ema.module if ema is not None else model
    va_acc, va_logits = validate(eval_model, dl_va)
    oof_preds[va_idx] = va_logits

    t_logits = infer_test(eval_model, test_dl)
    test_logits_accum += t_logits

    del model, ema, optimizer, scaler, ds_tr, ds_va, dl_tr, dl_va, eval_model, test_ds, test_dl
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print(f'Fold {fold_idx} done | best_va_acc {best_acc:.4f} | fold_time {(time.time()-t_fold)/60:.1f}m', flush=True)

    if fold_idx >= MAX_FOLDS:
        break

print(f'All folds done in {(time.time()-t_start_all)/60:.1f}m')

np.save('oof_logits.npy', oof_preds)
np.save('test_logits.npy', test_logits_accum)

test_probs = test_logits_accum / max(1, MAX_FOLDS)
test_pred = test_probs.argmax(1)
sub = pd.DataFrame({'image_id': test_files['image_id'], 'label': test_pred.astype(int)})
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv')
sub.head()

===== Fold 1/2 | train 9360 | valid 9361 =====


cfg: {'input_size': (3, 512, 512), 'crop_pct': 0.904, 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225)}
Majority class=3 | majority baseline acc on val=0.6154


val batch mean: [-0.1807, 0.3099, -0.4598]
val batch std : [1.05, 1.1089, 0.9553]


  it 50/390 | loss 1.7315 | acc 0.4433 | elapsed 13.0s


  it 100/390 | loss 1.4343 | acc 0.5437 | elapsed 25.2s


...

Fold 1 | Epoch 3/3 | tr_loss 0.6917 tr_acc 0.8062 | va_acc 0.7744 | elapsed_fold 6.2m


val preds[:16]: [0, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2]
val labs [:16]: [0, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 4]


Fold 1 done | best_va_acc 0.7744 | fold_time 6.9m


All folds done in 6.9m
Wrote submission.csv


,image_id,label
0,1234294272.jpg,3
1,1234332763.jpg,3
2,1234375577.jpg,2
3,1234555380.jpg,2
4,1234571117.jpg,3


In [18]:
# Full 5-fold training: tf_efficientnet_b3_ns @512 with Mixup/CutMix + EMA + cosine schedule (12 epochs)
import os, gc, math, time, warnings, numpy as np, pandas as pd, torch, timm
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageOps
from timm.utils import ModelEmaV3
from timm.data import Mixup
from timm.loss import SoftTargetCrossEntropy

warnings.filterwarnings('ignore', category=UserWarning)
torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision('high')
except Exception:
    pass

# Hyperparams per expert advice
MODEL_NAME = 'tf_efficientnet_b3_ns'
IMG_SIZE = 512
NUM_CLASSES = 5
FOLDS = 5
EPOCHS = 12
BATCH_SIZE = 24
BASE_LR = 2e-4
WD = 1e-4
EMA_DECAY = 0.9998
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)
NUM_WORKERS = min(8, os.cpu_count() or 2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CassavaDS(Dataset):
    def __init__(self, df, img_dir, transforms=None, is_test=False):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transforms = transforms
        self.is_test = is_test
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.img_dir, row['image_id'])
        with Image.open(path) as im:
            im = ImageOps.exif_transpose(im).convert('RGB')
        img = im if self.transforms is None else self.transforms(im)
        if self.is_test:
            return img, row['image_id']
        return img, torch.tensor(int(row['label']), dtype=torch.long)

def get_model_and_transforms():
    model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=NUM_CLASSES).to(device)
    cfg = timm.data.resolve_data_config({}, model=model)
    cfg['input_size'] = (3, IMG_SIZE, IMG_SIZE)
    cfg['mean'] = IMAGENET_MEAN
    cfg['std'] = IMAGENET_STD
    train_tfms = timm.data.create_transform(is_training=True, **cfg, auto_augment='rand-m9-mstd0.5-inc1', re_prob=0.25, re_mode='pixel')
    valid_tfms = timm.data.create_transform(is_training=False, **cfg)
    print('cfg:', {'input_size': cfg['input_size'], 'crop_pct': float(cfg.get('crop_pct', 0.875)), 'interpolation': cfg.get('interpolation','bicubic'), 'mean': cfg['mean'], 'std': cfg['std']})
    return model, train_tfms, valid_tfms

def validate(model_eval, loader):
    model_eval.eval()
    correct, total = 0, 0
    logits_all = []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model_eval(imgs)
            logits_all.append(logits.float().cpu().numpy())
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += imgs.size(0)
    return correct / max(1, total), np.concatenate(logits_all, axis=0)

def infer_test(model_eval, loader):
    model_eval.eval()
    logits_all = []
    with torch.no_grad():
        for imgs, _ids in loader:
            imgs = imgs.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model_eval(imgs)
            logits_all.append(logits.float().cpu().numpy())
    return np.concatenate(logits_all, axis=0)

def run_training():
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    df_shuf = df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)
    test_df = pd.DataFrame({'image_id': sorted([p.name for p in test_dir.glob('*.jpg')])})
    oof_logits = np.zeros((len(df_shuf), NUM_CLASSES), dtype=np.float32)
    test_logits_sum = np.zeros((len(test_df), NUM_CLASSES), dtype=np.float32)

    fold = 0
    start_all = time.time()
    for tr_idx, va_idx in skf.split(df_shuf['image_id'], df_shuf['label']):
        fold += 1
        t_fold = time.time()
        print(f'===== Fold {fold}/{FOLDS} | train {len(tr_idx)} | valid {len(va_idx)} =====', flush=True)
        df_tr = df_shuf.iloc[tr_idx].reset_index(drop=True)
        df_va = df_shuf.iloc[va_idx].reset_index(drop=True)

        model, train_tfms, valid_tfms = get_model_and_transforms()
        ds_tr = CassavaDS(df_tr, str(train_dir), transforms=train_tfms)
        ds_va = CassavaDS(df_va, str(train_dir), transforms=valid_tfms)
        dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True, persistent_workers=False)
        dl_va = DataLoader(ds_va, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)
        test_ds = CassavaDS(test_df, str(test_dir), transforms=valid_tfms, is_test=True)
        test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)

        # Mixup/CutMix + SoftTarget CE
        mixup_fn = Mixup(mixup_alpha=0.4, cutmix_alpha=1.0, prob=0.5, switch_prob=0.5, mode='batch', label_smoothing=0.0, num_classes=NUM_CLASSES)
        criterion = SoftTargetCrossEntropy()
        optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WD)
        scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())
        ema = ModelEmaV3(model, decay=EMA_DECAY)

        total_steps = EPOCHS * len(dl_tr)
        warmup_steps = len(dl_tr)  # 1 epoch warmup
        global_step = 0
        best_acc = 0.0
        best_state = None

        for epoch in range(1, EPOCHS+1):
            model.train()
            running_loss = 0.0; correct = 0; total = 0; t0 = time.time()
            # Optionally turn off mixup last 2 epochs to sharpen
            use_mix = epoch <= EPOCHS - 2
            for it, (imgs, labels) in enumerate(dl_tr):
                step = global_step + it
                if step < warmup_steps:
                    lr_scale = step / max(1, warmup_steps)
                else:
                    progress = (step - warmup_steps) / max(1, (total_steps - warmup_steps))
                    lr_scale = 0.5 * (1 + math.cos(math.pi * progress))
                for pg in optimizer.param_groups:
                    pg['lr'] = BASE_LR * lr_scale

                imgs = imgs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                if use_mix and mixup_fn is not None:
                    imgs, targets = mixup_fn(imgs, labels)
                else:
                    # one-hot targets for SoftTarget CE when mixup disabled
                    targets = torch.zeros((labels.size(0), NUM_CLASSES), device=labels.device)
                    targets.scatter_(1, labels.unsqueeze(1), 1.0)

                optimizer.zero_grad(set_to_none=True)
                with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                    logits = model(imgs)
                    loss = criterion(logits, targets)
                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                if ema is not None:
                    ema.update(model)

                running_loss += loss.item() * imgs.size(0)
                with torch.no_grad():
                    preds = logits.argmax(1)
                    correct += (preds == labels).sum().item()
                    total += imgs.size(0)
                if (it+1) % 50 == 0:
                    print(f'  it {it+1}/{len(dl_tr)} | loss {running_loss/max(1,total):.4f} | acc {correct/max(1,total):.4f} | elapsed {time.time()-t0:.1f}s', flush=True)

            global_step += len(dl_tr)
            eval_model = ema.module if ema is not None else model
            va_acc, _ = validate(eval_model, dl_va)
            print(f'Fold {fold} | Epoch {epoch}/{EPOCHS} | tr_loss {running_loss/max(1,total):.4f} tr_acc {correct/max(1,total):.4f} | va_acc {va_acc:.4f} | elapsed_fold {(time.time()-t_fold)/60:.1f}m', flush=True)
            if va_acc > best_acc:
                best_acc = va_acc
                best_state = {k: v.cpu() for k, v in eval_model.state_dict().items()}

        # Load best EMA state
        if best_state is not None:
            best_model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=NUM_CLASSES).to(device)
            best_model.load_state_dict(best_state, strict=True)
        else:
            best_model = ema.module if ema is not None else model

        # OOF fill
        va_acc, va_logits = validate(best_model, dl_va)
        oof_logits[va_idx] = va_logits

        # Test inference accumulate
        t_logits = infer_test(best_model, test_dl)
        test_logits_sum += t_logits

        # Save best checkpoint
        ckpt_path = f'b3_fold{fold}_best.pth'
        torch.save({'state_dict': best_state, 'best_va_acc': best_acc}, ckpt_path)
        print(f'Fold {fold} done | best_va_acc {best_acc:.4f} | ckpt {ckpt_path} | fold_time {(time.time()-t_fold)/60:.1f}m', flush=True)

        # Cleanup
        del model, ema, optimizer, scaler, ds_tr, ds_va, dl_tr, dl_va, best_model, test_ds, test_dl
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    print(f'All folds done in {(time.time()-start_all)/60:.1f}m')

    # Compute OOF accuracy
    oof_pred = oof_logits.argmax(1)
    oof_acc = (oof_pred == df_shuf['label'].values).mean()
    print(f'OOF accuracy: {oof_acc:.5f}')

    # Save artifacts
    np.save('oof_logits_b3.npy', oof_logits)
    np.save('test_logits_b3.npy', test_logits_sum)

    # Build a quick submission from averaged test logits (no TTA yet, 5-fold avg)
    test_pred = test_logits_sum.argmax(1)
    sub = pd.DataFrame({'image_id': test_df['image_id'], 'label': test_pred.astype(int)})
    sub.to_csv('submission_b3.csv', index=False)
    print('Wrote submission_b3.csv')

run_training()

===== Fold 1/5 | train 14976 | valid 3745 =====


cfg: {'input_size': (3, 512, 512), 'crop_pct': 0.904, 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225)}


  it 50/624 | loss 1.9825 | acc 0.2775 | elapsed 12.9s


  it 100/624 | loss 1.6974 | acc 0.4058 | elapsed 25.1s


  it 150/624 | loss 1.5368 | acc 0.4661 | elapsed 37.4s


...

  it 550/624 | loss 0.3588 | acc 0.8758 | elapsed 137.8s


  it 600/624 | loss 0.3569 | acc 0.8757 | elapsed 150.2s


Fold 5 | Epoch 12/12 | tr_loss 0.3561 tr_acc 0.8757 | va_acc 0.8865 | elapsed_fold 33.6m


Fold 5 done | best_va_acc 0.8865 | ckpt b3_fold5_best.pth | fold_time 34.0m


All folds done in 169.8m
OOF accuracy: 0.88617
Wrote submission_b3.csv


In [23]:
# Full 5-fold training: convnext_base.fb_in22k_ft_in1k @512 with Mixup/CutMix + EMA + cosine schedule (12 epochs)
import os, gc, math, time, warnings, numpy as np, pandas as pd, torch, timm
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader
from PIL import Image, ImageOps
from timm.utils import ModelEmaV3
from timm.data import Mixup
from timm.loss import SoftTargetCrossEntropy

warnings.filterwarnings('ignore', category=UserWarning)
torch.backends.cudnn.benchmark = True
try:
    torch.set_float32_matmul_precision('high')
except Exception:
    pass

# Hyperparams per expert advice
MODEL_NAME_CNX = 'convnext_base.fb_in22k_ft_in1k'
IMG_SIZE = 512
NUM_CLASSES = 5
FOLDS = 5
EPOCHS = 12
BATCH_SIZE = 20  # convnext is heavier; adjust if OOM
BASE_LR = 8e-4
WD = 0.05
EMA_DECAY = 0.9998
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)
NUM_WORKERS = min(8, os.cpu_count() or 2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CassavaDSConv(Dataset):
    def __init__(self, df, img_dir, transforms=None, is_test=False):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transforms = transforms
        self.is_test = is_test
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.img_dir, row['image_id'])
        with Image.open(path) as im:
            im = ImageOps.exif_transpose(im).convert('RGB')
        img = im if self.transforms is None else self.transforms(im)
        if self.is_test:
            return img, row['image_id']
        return img, torch.tensor(int(row['label']), dtype=torch.long)

def get_model_and_transforms_convnext():
    model = timm.create_model(MODEL_NAME_CNX, pretrained=True, num_classes=NUM_CLASSES).to(device)
    cfg = timm.data.resolve_data_config({}, model=model)
    cfg['input_size'] = (3, IMG_SIZE, IMG_SIZE)
    cfg['mean'] = IMAGENET_MEAN
    cfg['std'] = IMAGENET_STD
    train_tfms = timm.data.create_transform(is_training=True, **cfg, auto_augment='rand-m9-mstd0.5-inc1', re_prob=0.25, re_mode='pixel')
    valid_tfms = timm.data.create_transform(is_training=False, **cfg)
    print('CNX cfg:', {'input_size': cfg['input_size'], 'crop_pct': float(cfg.get('crop_pct', 0.875)), 'interpolation': cfg.get('interpolation','bicubic'), 'mean': cfg['mean'], 'std': cfg['std']})
    return model, train_tfms, valid_tfms

def validate(model_eval, loader):
    model_eval.eval()
    correct, total = 0, 0
    logits_all = []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model_eval(imgs)
            logits_all.append(logits.float().cpu().numpy())
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += imgs.size(0)
    return correct / max(1, total), np.concatenate(logits_all, axis=0)

def infer_test(model_eval, loader):
    model_eval.eval()
    logits_all = []
    with torch.no_grad():
        for imgs, _ids in loader:
            imgs = imgs.to(device, non_blocking=True)
            with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                logits = model_eval(imgs)
            logits_all.append(logits.float().cpu().numpy())
    return np.concatenate(logits_all, axis=0)

def run_training_convnext():
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    df_shuf = df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)
    test_df = pd.DataFrame({'image_id': sorted([p.name for p in test_dir.glob('*.jpg')])})
    oof_logits = np.zeros((len(df_shuf), NUM_CLASSES), dtype=np.float32)
    test_logits_sum = np.zeros((len(test_df), NUM_CLASSES), dtype=np.float32)

    fold = 0
    start_all = time.time()
    for tr_idx, va_idx in skf.split(df_shuf['image_id'], df_shuf['label']):
        fold += 1
        t_fold = time.time()
        print(f'===== ConvNeXt Fold {fold}/{FOLDS} | train {len(tr_idx)} | valid {len(va_idx)} =====', flush=True)
        df_tr = df_shuf.iloc[tr_idx].reset_index(drop=True)
        df_va = df_shuf.iloc[va_idx].reset_index(drop=True)

        model, train_tfms, valid_tfms = get_model_and_transforms_convnext()
        ds_tr = CassavaDSConv(df_tr, str(train_dir), transforms=train_tfms)
        ds_va = CassavaDSConv(df_va, str(train_dir), transforms=valid_tfms)
        dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True, persistent_workers=False)
        dl_va = DataLoader(ds_va, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)
        test_ds = CassavaDSConv(test_df, str(test_dir), transforms=valid_tfms, is_test=True)
        test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=False)

        mixup_fn = Mixup(mixup_alpha=0.4, cutmix_alpha=1.0, prob=0.5, switch_prob=0.5, mode='batch', label_smoothing=0.0, num_classes=NUM_CLASSES)
        criterion = SoftTargetCrossEntropy()
        optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WD)
        scaler = torch.amp.GradScaler('cuda', enabled=torch.cuda.is_available())
        ema = ModelEmaV3(model, decay=EMA_DECAY)

        total_steps = EPOCHS * len(dl_tr)
        warmup_steps = len(dl_tr)  # 1 epoch warmup
        global_step = 0
        best_acc = 0.0
        best_state = None

        for epoch in range(1, EPOCHS+1):
            model.train()
            running_loss = 0.0; correct = 0; total = 0; t0 = time.time()
            use_mix = epoch <= EPOCHS - 2
            for it, (imgs, labels) in enumerate(dl_tr):
                step = global_step + it
                if step < warmup_steps:
                    lr_scale = step / max(1, warmup_steps)
                else:
                    progress = (step - warmup_steps) / max(1, (total_steps - warmup_steps))
                    lr_scale = 0.5 * (1 + math.cos(math.pi * progress))
                for pg in optimizer.param_groups:
                    pg['lr'] = BASE_LR * lr_scale

                imgs = imgs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                if use_mix and mixup_fn is not None:
                    imgs, targets = mixup_fn(imgs, labels)
                else:
                    targets = torch.zeros((labels.size(0), NUM_CLASSES), device=labels.device)
                    targets.scatter_(1, labels.unsqueeze(1), 1.0)

                optimizer.zero_grad(set_to_none=True)
                with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                    logits = model(imgs)
                    loss = criterion(logits, targets)
                scaler.scale(loss).backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                if ema is not None:
                    ema.update(model)

                running_loss += loss.item() * imgs.size(0)
                with torch.no_grad():
                    preds = logits.argmax(1)
                    correct += (preds == labels).sum().item()
                    total += imgs.size(0)
                if (it+1) % 50 == 0:
                    print(f'  it {it+1}/{len(dl_tr)} | loss {running_loss/max(1,total):.4f} | acc {correct/max(1,total):.4f} | elapsed {time.time()-t0:.1f}s', flush=True)

            global_step += len(dl_tr)
            eval_model = ema.module if ema is not None else model
            va_acc, _ = validate(eval_model, dl_va)
            print(f'ConvNeXt Fold {fold} | Epoch {epoch}/{EPOCHS} | tr_loss {running_loss/max(1,total):.4f} tr_acc {correct/max(1,total):.4f} | va_acc {va_acc:.4f} | elapsed_fold {(time.time()-t_fold)/60:.1f}m', flush=True)
            if va_acc > best_acc:
                best_acc = va_acc
                best_state = {k: v.cpu() for k, v in eval_model.state_dict().items()}

        if best_state is not None:
            best_model = timm.create_model(MODEL_NAME_CNX, pretrained=False, num_classes=NUM_CLASSES).to(device)
            best_model.load_state_dict(best_state, strict=True)
        else:
            best_model = ema.module if ema is not None else model

        va_acc, va_logits = validate(best_model, dl_va)
        oof_logits[va_idx] = va_logits
        t_logits = infer_test(best_model, test_dl)
        test_logits_sum += t_logits

        ckpt_path = f'convnext_fold{fold}_best.pth'
        torch.save({'state_dict': best_state, 'best_va_acc': best_acc}, ckpt_path)
        print(f'ConvNeXt Fold {fold} done | best_va_acc {best_acc:.4f} | ckpt {ckpt_path} | fold_time {(time.time()-t_fold)/60:.1f}m', flush=True)

        del model, ema, optimizer, scaler, ds_tr, ds_va, dl_tr, dl_va, best_model, test_ds, test_dl
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    print(f'ConvNeXt all folds done in {(time.time()-start_all)/60:.1f}m')
    oof_pred = oof_logits.argmax(1)
    oof_acc = (oof_pred == df_shuf['label'].values).mean()
    print(f'ConvNeXt OOF accuracy: {oof_acc:.5f}')
    np.save('oof_logits_convnext.npy', oof_logits)
    np.save('test_logits_convnext.npy', test_logits_sum)
    test_pred = test_logits_sum.argmax(1)
    sub = pd.DataFrame({'image_id': test_df['image_id'], 'label': test_pred.astype(int)})
    sub.to_csv('submission_convnext.csv', index=False)
    print('Wrote submission_convnext.csv')

# To run after B3 completes: run_training_convnext()

In [27]:
# TTA inference and ensembling helpers (B3 4x TTA; optional blend later)
import os, numpy as np, pandas as pd, torch, timm, time
from pathlib import Path
from PIL import Image, ImageOps
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

class TestTTADS(Dataset):
    def __init__(self, img_dir, files, tfms):
        self.img_dir = Path(img_dir)
        self.files = files
        self.tfms = tfms
    def __len__(self): return len(self.files)
    def __getitem__(self, idx):
        fname = self.files[idx]
        with Image.open(self.img_dir / fname) as im:
            im = ImageOps.exif_transpose(im).convert('RGB')
        return im, fname

def collate_pil(batch):
    # Collate a batch of (PIL.Image, filename) into lists to avoid default_collate errors
    imgs = [b[0] for b in batch]
    names = [b[1] for b in batch]
    return imgs, names

def build_eval_tfms_for(model_name='tf_efficientnet_b3_ns', img_size=512):
    model = timm.create_model(model_name, pretrained=False, num_classes=5).to(device)
    cfg = timm.data.resolve_data_config({}, model=model)
    cfg['input_size'] = (3, img_size, img_size)
    cfg['mean'] = IMAGENET_MEAN
    cfg['std']  = IMAGENET_STD
    tfms = timm.data.create_transform(is_training=False, **cfg)
    return tfms, cfg

@torch.no_grad()
def infer_tta_b3(checkpoints, img_dir='test_images', batch_size=32, img_size=512):
    files = sorted([p.name for p in Path(img_dir).glob('*.jpg')])
    tfms, cfg = build_eval_tfms_for('tf_efficientnet_b3_ns', img_size=img_size)
    ds = TestTTADS(img_dir, files, tfms)
    dl = DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,  # avoid MP issues with PIL + custom collate
        pin_memory=True,
        collate_fn=collate_pil
    )

    def apply_flip(img, flip_id):
        # 0: none, 1: hflip, 2: vflip, 3: hvflip
        if flip_id == 1: img = F.hflip(img)
        elif flip_id == 2: img = F.vflip(img)
        elif flip_id == 3: img = F.vflip(F.hflip(img))
        return img

    all_logits = np.zeros((len(files), 5), dtype=np.float32)
    for ckpt_path in checkpoints:
        if not Path(ckpt_path).exists():
            print(f'Skip missing checkpoint {ckpt_path}')
            continue
        model = timm.create_model('tf_efficientnet_b3_ns', pretrained=False, num_classes=5).to(device)
        sd = torch.load(ckpt_path, map_location='cpu')['state_dict']
        model.load_state_dict(sd, strict=True)
        model.eval()
        fold_logits = np.zeros_like(all_logits)
        ptr = 0
        t0 = time.time()
        for imgs_pil, names in dl:
            bs = len(imgs_pil)
            logits_sum = torch.zeros((bs, 5), device=device)
            for f in range(4):
                batch = [apply_flip(img, f) for img in imgs_pil]
                batch = torch.stack([ds.tfms(b) for b in batch]).to(device, non_blocking=True)
                with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                    logits = model(batch)
                logits_sum += logits.float()
            logits_avg = (logits_sum / 4.0).cpu().numpy()
            fold_logits[ptr:ptr+bs] = logits_avg
            ptr += bs
        all_logits += fold_logits
        print(f'Inferred {ckpt_path} in {time.time()-t0:.1f}s')
        del model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    all_logits /= max(1, len(checkpoints))
    return files, all_logits

# Generic TTA for any timm model (e.g., convnext_base)
@torch.no_grad()
def infer_tta_model(model_name, checkpoints, img_dir='test_images', batch_size=32, img_size=512):
    files = sorted([p.name for p in Path(img_dir).glob('*.jpg')])
    tfms, cfg = build_eval_tfms_for(model_name, img_size=img_size)
    ds = TestTTADS(img_dir, files, tfms)
    dl = DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=collate_pil
    )

    def apply_flip(img, flip_id):
        if flip_id == 1: img = F.hflip(img)
        elif flip_id == 2: img = F.vflip(img)
        elif flip_id == 3: img = F.vflip(F.hflip(img))
        return img

    all_logits = np.zeros((len(files), 5), dtype=np.float32)
    for ckpt_path in checkpoints:
        if not Path(ckpt_path).exists():
            print(f'Skip missing checkpoint {ckpt_path}')
            continue
        model = timm.create_model(model_name, pretrained=False, num_classes=5).to(device)
        sd = torch.load(ckpt_path, map_location='cpu')['state_dict']
        model.load_state_dict(sd, strict=True)
        model.eval()
        fold_logits = np.zeros_like(all_logits)
        ptr = 0
        t0 = time.time()
        for imgs_pil, names in dl:
            bs = len(imgs_pil)
            logits_sum = torch.zeros((bs, 5), device=device)
            for f in range(4):
                batch = [apply_flip(img, f) for img in imgs_pil]
                batch = torch.stack([ds.tfms(b) for b in batch]).to(device, non_blocking=True)
                with torch.amp.autocast('cuda', enabled=torch.cuda.is_available()):
                    logits = model(batch)
                logits_sum += logits.float()
            logits_avg = (logits_sum / 4.0).cpu().numpy()
            fold_logits[ptr:ptr+bs] = logits_avg
            ptr += bs
        all_logits += fold_logits
        print(f'Inferred {ckpt_path} in {time.time()-t0:.1f}s')
        del model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    all_logits /= max(1, len(checkpoints))
    return files, all_logits

def write_submission_from_logits(files, logits, out_csv):
    pred = logits.argmax(1)
    sub = pd.DataFrame({'image_id': files, 'label': pred.astype(int)})
    sub.to_csv(out_csv, index=False)
    print(f'Wrote {out_csv}')

# If B3 folds finished, you can run:
# files, logits_b3_tta = infer_tta_b3([f'b3_fold{i}_best.pth' for i in range(1,6)], img_size=512, batch_size=24)
# np.save('test_logits_b3_tta.npy', logits_b3_tta)
# write_submission_from_logits(files, logits_b3_tta, 'submission_b3_tta.csv')

# Optional simple blend if both model logits exist:
def try_blend_and_write():
    test_df = pd.DataFrame({'image_id': sorted([p.name for p in Path('test_images').glob('*.jpg')])})
    paths = {'b3': 'test_logits_b3.npy', 'cnx': 'test_logits_convnext.npy', 'b3tta': 'test_logits_b3_tta.npy'}
    logits_list = []
    names = []
    for name, p in paths.items():
        if Path(p).exists():
            logits = np.load(p)
            logits_list.append(logits)
            names.append(name)
    if len(logits_list) == 0:
        print('No logits found to blend yet.')
        return
    # Equal-weight blend
    blend = np.mean(np.stack(logits_list, axis=0), axis=0)
    pred = blend.argmax(1)
    sub = pd.DataFrame({'image_id': test_df['image_id'], 'label': pred.astype(int)})
    sub.to_csv('submission_ensemble.csv', index=False)
    print(f'Wrote submission_ensemble.csv from components: {names}')

# After both runs, call try_blend_and_write() to produce ensemble submission.

In [22]:
# Run 4-flip TTA for B3 folds and write submission
checkpoints = [f'b3_fold{i}_best.pth' for i in range(1,6)]
print('Running B3 4x TTA on checkpoints:', checkpoints, flush=True)
files, logits_b3_tta = infer_tta_b3(checkpoints, img_dir='test_images', batch_size=24, img_size=512)
np.save('test_logits_b3_tta.npy', logits_b3_tta)
write_submission_from_logits(files, logits_b3_tta, 'submission_b3_tta.csv')
print('B3 TTA done. logits shape:', logits_b3_tta.shape)

Running B3 4x TTA on checkpoints: ['b3_fold1_best.pth', 'b3_fold2_best.pth', 'b3_fold3_best.pth', 'b3_fold4_best.pth', 'b3_fold5_best.pth']


Inferred b3_fold1_best.pth in 111.1s


Inferred b3_fold2_best.pth in 106.1s


Inferred b3_fold3_best.pth in 106.3s


Inferred b3_fold4_best.pth in 105.6s


Inferred b3_fold5_best.pth in 106.6s
Wrote submission_b3_tta.csv
B3 TTA done. logits shape: (2676, 5)


In [24]:
# Safety baseline submit + start ConvNeXt training
import os, shutil
if os.path.exists('submission_b3_tta.csv'):
    shutil.copyfile('submission_b3_tta.csv', 'submission.csv')
    print('Copied submission_b3_tta.csv -> submission.csv')
else:
    print('WARNING: submission_b3_tta.csv not found; baseline not copied')

print('Starting ConvNeXt-Base 5-fold training...')
run_training_convnext()

Copied submission_b3_tta.csv -> submission.csv
Starting ConvNeXt-Base 5-fold training...
===== ConvNeXt Fold 1/5 | train 14976 | valid 3745 =====


CNX cfg: {'input_size': (3, 512, 512), 'crop_pct': 0.875, 'interpolation': 'bicubic', 'mean': (0.485, 0.456, 0.406), 'std': (0.229, 0.224, 0.225)}


  it 50/748 | loss 1.1595 | acc 0.5960 | elapsed 47.7s


  it 100/748 | loss 1.0482 | acc 0.6245 | elapsed 68.7s


  it 150/748 | loss 0.9415 | acc 0.6573 | elapsed 89.7s


...

  it 650/748 | loss 0.3196 | acc 0.8913 | elapsed 280.4s


  it 700/748 | loss 0.3195 | acc 0.8906 | elapsed 301.8s


ConvNeXt Fold 5 | Epoch 12/12 | tr_loss 0.3176 tr_acc 0.8918 | va_acc 0.9060 | elapsed_fold 70.1m


ConvNeXt Fold 5 done | best_va_acc 0.9060 | ckpt convnext_fold5_best.pth | fold_time 70.9m


ConvNeXt all folds done in 354.6m
ConvNeXt OOF accuracy: 0.89851
Wrote submission_convnext.csv


In [25]:
# After ConvNeXt training finishes: run 4-flip TTA for ConvNeXt and blend with B3 TTA
from pathlib import Path
import numpy as np, pandas as pd

def run_convnext_tta_and_blend():
    cnx_ckpts = [f'convnext_fold{i}_best.pth' for i in range(1,6)]
    missing = [p for p in cnx_ckpts if not Path(p).exists()]
    if missing:
        print('ConvNeXt checkpoints not all present yet, missing:', missing)
        return
    print('Running ConvNeXt 4x TTA on checkpoints:', cnx_ckpts, flush=True)
    files, logits_cnx_tta = infer_tta_model('convnext_base.fb_in22k_ft_in1k', cnx_ckpts, img_dir='test_images', batch_size=20, img_size=512)
    np.save('test_logits_convnext_tta.npy', logits_cnx_tta)
    write_submission_from_logits(files, logits_cnx_tta, 'submission_convnext_tta.csv')
    print('ConvNeXt TTA done. logits shape:', logits_cnx_tta.shape)

    # Equal-weight blend with B3 TTA if available
    b3_path = Path('test_logits_b3_tta.npy')
    cnx_path = Path('test_logits_convnext_tta.npy')
    if b3_path.exists() and cnx_path.exists():
        logits_b3 = np.load(b3_path)
        logits_cnx = np.load(cnx_path)
        if logits_b3.shape != logits_cnx.shape:
            print('Shape mismatch in logits, cannot blend:', logits_b3.shape, logits_cnx.shape)
            return
        blend = 0.5 * logits_b3 + 0.5 * logits_cnx
        pred = blend.argmax(1)
        sub = pd.DataFrame({'image_id': files, 'label': pred.astype(int)})
        sub.to_csv('submission_ensemble.csv', index=False)
        print('Wrote submission_ensemble.csv (equal-weight B3_TTA + ConvNeXt_TTA)')
    else:
        print('Missing logits for blending: b3_tta or convnext_tta not found.')

# Call run_convnext_tta_and_blend() after ConvNeXt training completes.

In [28]:
# Run ConvNeXt TTA and blend; set final submission.csv
import os, shutil
print('Starting ConvNeXt TTA + blend...', flush=True)
run_convnext_tta_and_blend()

chosen = None
if os.path.exists('submission_ensemble.csv'):
    shutil.copyfile('submission_ensemble.csv', 'submission.csv')
    chosen = 'submission_ensemble.csv'
elif os.path.exists('submission_convnext_tta.csv'):
    shutil.copyfile('submission_convnext_tta.csv', 'submission.csv')
    chosen = 'submission_convnext_tta.csv'
elif os.path.exists('submission_b3_tta.csv'):
    shutil.copyfile('submission_b3_tta.csv', 'submission.csv')
    chosen = 'submission_b3_tta.csv'
print('submission.csv set from:', chosen)

Starting ConvNeXt TTA + blend...


Running ConvNeXt 4x TTA on checkpoints: ['convnext_fold1_best.pth', 'convnext_fold2_best.pth', 'convnext_fold3_best.pth', 'convnext_fold4_best.pth', 'convnext_fold5_best.pth']


Inferred convnext_fold1_best.pth in 111.8s


Inferred convnext_fold2_best.pth in 113.4s


Inferred convnext_fold3_best.pth in 110.7s


Inferred convnext_fold4_best.pth in 110.9s


Inferred convnext_fold5_best.pth in 110.9s
Wrote submission_convnext_tta.csv
ConvNeXt TTA done. logits shape: (2676, 5)
Wrote submission_ensemble.csv (equal-weight B3_TTA + ConvNeXt_TTA)
submission.csv set from: submission_ensemble.csv


In [29]:
# Optimize ensemble weight on OOF logits and apply to TTA test logits
import numpy as np, pandas as pd, os, shutil
from pathlib import Path

def optimize_and_build_weighted_submission():
    # Load OOF logits
    b3_oof = np.load('oof_logits_b3.npy') if Path('oof_logits_b3.npy').exists() else None
    cnx_oof = np.load('oof_logits_convnext.npy') if Path('oof_logits_convnext.npy').exists() else None
    if b3_oof is None or cnx_oof is None:
        print('Missing OOF logits for optimization.'); return
    # Recreate df_shuf order to get y_true in the same order
    df_shuf = df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)
    y_true = df_shuf['label'].values.astype(int)
    assert b3_oof.shape == cnx_oof.shape == (len(y_true), 5), f'OOF shape mismatch: b3 {b3_oof.shape}, cnx {cnx_oof.shape}, y {len(y_true)}'

    # Grid search weight w for ConvNeXt in [0.4, 0.6]
    best_w, best_acc = 0.5, 0.0
    for w in np.linspace(0.4, 0.6, 21):
        blend = w * cnx_oof + (1.0 - w) * b3_oof
        pred = blend.argmax(1)
        acc = (pred == y_true).mean()
        if acc > best_acc:
            best_acc, best_w = acc, float(w)
    print(f'Best OOF blend: w={best_w:.2f} (ConvNeXt weight), OOF acc={best_acc:.5f}')

    # Apply to test TTA logits
    b3_test = np.load('test_logits_b3_tta.npy') if Path('test_logits_b3_tta.npy').exists() else None
    cnx_test = np.load('test_logits_convnext_tta.npy') if Path('test_logits_convnext_tta.npy').exists() else None
    if b3_test is None or cnx_test is None:
        print('Missing test TTA logits for blending.'); return
    assert b3_test.shape == cnx_test.shape, f'Test shape mismatch: {b3_test.shape} vs {cnx_test.shape}'
    blend_test = best_w * cnx_test + (1.0 - best_w) * b3_test
    test_files = sorted([p.name for p in Path('test_images').glob('*.jpg')])
    sub = pd.DataFrame({'image_id': test_files, 'label': blend_test.argmax(1).astype(int)})
    out_csv = 'submission_weighted_ensemble.csv'
    sub.to_csv(out_csv, index=False)
    print(f'Wrote {out_csv} with w={best_w:.2f}')
    shutil.copyfile(out_csv, 'submission.csv')
    print('submission.csv updated ->', out_csv)

# Run optimization and build weighted submission
optimize_and_build_weighted_submission()

Best OOF blend: w=0.58 (ConvNeXt weight), OOF acc=0.89883
Wrote submission_weighted_ensemble.csv with w=0.58
submission.csv updated -> submission_weighted_ensemble.csv


In [30]:
# Set submission to ConvNeXt TTA-only per expert advice
import shutil, os
src = 'submission_convnext_tta.csv'
dst = 'submission.csv'
if os.path.exists(src):
    shutil.copyfile(src, dst)
    print(f'Copied {src} -> {dst}')
else:
    print(f'MISSING: {src}')

Copied submission_convnext_tta.csv -> submission.csv
